In [1]:
pip install osmnx networkx


In [2]:
!pip install openrouteservice folium 


In [9]:
pip install scikit-learn 


In [10]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [1]:
import osmnx as ox
import networkx as nx
import numpy as np

# Step 1: Get drivable street network of Dehradun
G = ox.graph_from_place("Dehradun, India", network_type="drive")

# Step 2: Geocode start and end places
start_address = "Paltan Bazar, Dehradun, India"
end_address = "Graphic Era Hill University, Dehradun, India"

start_point = ox.geocoder.geocode(start_address)
end_point = ox.geocoder.geocode(end_address)

# Step 3: Get nearest graph nodes to the start and end points
orig_node = ox.nearest_nodes(G, start_point[1], start_point[0])
dest_node = ox.nearest_nodes(G, end_point[1], end_point[0])

# Step 4: Define A* Euclidean heuristic
def euclidean_heuristic(u, v):
    u_point = (G.nodes[u]['y'], G.nodes[u]['x'])  # lat, lon
    v_point = (G.nodes[v]['y'], G.nodes[v]['x'])
    return np.linalg.norm(np.array(u_point) - np.array(v_point))

# Step 5: Run A* to get the shortest-distance route
route = nx.astar_path(G, orig_node, dest_node, heuristic=euclidean_heuristic, weight='length')

# Step 6: Convert route to lat/lon coordinates
route_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in route]

# Step 7: Generate Google Maps directions link (free, no API key)
base_url = "https://www.google.com/maps/dir/?api=1"
origin = f"{route_coords[0][0]},{route_coords[0][1]}"
destination = f"{route_coords[-1][0]},{route_coords[-1][1]}"
waypoints = "|".join([f"{lat},{lon}" for lat, lon in route_coords[1:-1][::30]])  # pick every 30th point

# Final URL
gmap_url = f"{base_url}&origin={origin}&destination={destination}&travelmode=driving"
if waypoints:
    gmap_url += f"&waypoints={waypoints}"

# Show the link
print("Open this link in your browser to see the route on Google Maps:")
print(gmap_url)


Open this link in your browser to see the route on Google Maps:
https://www.google.com/maps/dir/?api=1&origin=30.3203474,78.0374259&destination=30.2725465,77.9999065&travelmode=driving&waypoints=30.319598,78.0362048|30.3022295,78.010215|30.2846527,77.99765|30.2718416,77.998957


In [2]:
import osmnx as ox
import networkx as nx
import numpy as np
import folium

# Step 1: Load graph
G = ox.graph_from_place("Dehradun, India", network_type='drive')

# Step 2: Define locations
start = "Paltan Bazar, Dehradun, India"
end = "Graphic Era Hill University, Dehradun, India"

# Step 3: Geocode to get lat/lon
start_point = ox.geocode(start)
end_point = ox.geocode(end)

# Step 4: Find nearest graph nodes
orig_node = ox.nearest_nodes(G, X=start_point[1], Y=start_point[0])
dest_node = ox.nearest_nodes(G, X=end_point[1], Y=end_point[0])

# Step 5: Define A* heuristic (Euclidean)
def euclidean_heuristic(u, v):
    u_xy = (G.nodes[u]['x'], G.nodes[u]['y'])
    v_xy = (G.nodes[v]['x'], G.nodes[v]['y'])
    return np.linalg.norm(np.array(u_xy) - np.array(v_xy))

# Step 6: Get both paths
dijkstra_path = nx.shortest_path(G, orig_node, dest_node, weight='length')
astar_path = nx.astar_path(G, orig_node, dest_node, heuristic=euclidean_heuristic, weight='length')

# Step 7: Convert to lat/lon coordinates
dijkstra_coords = [(G.nodes[n]['y'], G.nodes[n]['x']) for n in dijkstra_path]
ast for n in astar_path]

# Step 8: Create Folium map centered at start
map_center = [(start_point[0] + end_point[0]) / 2, (start_point[1] + end_point[1]) / 2]
m = folium.Map(location=map_center, zoom_start=13)

# Add Dijkstra path (blue)
folium.PolyLine(dijkstra_coords, color='blue', weight=4, opacity=0.7, tooltip='Dijkstra Path').add_to(m)

# Add A* path (green)
folium='green', weight=4, opacity=0.7, tooltip='A* Path').add_to(m)

# Add markers
folium.Marker(start_point, popup='Start: Paltan Bazar', icon=folium.Icon(color='red')).add_to(m)
folium.Marker(end_point, popup='End: Graphic Era', icon=folium.Icon(color='purple')).add_to(m)

# Save and display
m.save("route_comparison_map.html")
print("✅ Map saved as 'route_comparison_map.html'. Open it in a browser to compare paths.")


✅ Map saved as 'route_comparison_map.html'. Open it in a browser to compare paths.
